# Performing Sentiment Analysis via Textblob and VADER

In [11]:
import pandas as pd
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [13]:
data = pd.concat(
    map(pd.read_csv, ['tweets_2022_03_05-2022_03_11.csv', 'tweets_2022_03_28-2022_04_04.csv']), ignore_index=True)
data.to_csv("tweets_2022_03_05-2022_03_11_and_2022_03_28-2022_04_04.csv")
data

,Unnamed: 0,Unnamed: 0.1,text,retweet_count,favorite_count,followers_count,verified,listed_count,created_at,hashtags,name
0,0,0.0,"""It is an open ledger, trying to sneak lots of...",151,520,2437101,True,10788,2022-03-05 09:33:07+00:00,"[{'text': 'crypto', 'indices': [61, 68]}]",Bitcoin News
1,1,1.0,“The #crypto market today has a market capital...,87,245,2437101,True,10788,2022-03-05 12:03:14+00:00,"[{'text': 'crypto', 'indices': [5, 12]}]",Bitcoin News
2,2,2.0,G7 countries and the EU are looking at ways to...,95,245,2437101,True,10788,2022-03-05 13:33:29+00:00,"[{'text': 'cryptocurrency', 'indices': [88, 10...",Bitcoin News
3,3,3.0,JUST IN: 🇸🇬 Singapore has introduced sanctions...,149,1043,708057,True,1449,2022-03-05 11:01:13+00:00,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",Watcher.Guru
4,4,4.0,Make sure you check in on your bros in the str...,45,149,7822,True,80,2022-03-05 14:45:25+00:00,[],Ian Heinisch
...,...,...,...,...,...,...,...,...,...,...,...
12712,766,NaN,🎁 #NFTGiveaway 🎁\n⚡️Ultra Rare Stud NFTs Comin...,483,608,176702,True,33,Mon Apr 04 13:16:59 +0000 2022,"[{'text': 'NFTGiveaway', 'indices': [2, 14]}, ...",LOVE
12713,767,NaN,#ARIES. Felicidades. Con el #Sol y #Mercurio e...,77,428,373077,True,414,Mon Apr 04 09:42:27 +0000 2022,"[{'text': 'ARIES', 'indices': [0, 6]}, {'text'...",Esperanza Gracia
12714,768,NaN,Get industry best interest % just for HODLing ...,24,69,41535,True,510,Mon Apr 04 07:00:29 +0000 2022,"[{'text': 'BTC', 'indices': [108, 112]}]",Wirex
12715,769,NaN,"anti-emperyalizmi unutan, #Sol olmaktan çıkan ...",11,40,46885,True,75,Mon Apr 04 15:20:52 +0000 2022,"[{'text': 'Sol', 'indices': [26, 30]}]",Şerafettin Turpcu


## Clean Tweets

In [14]:
text_data = data[['Unnamed: 0', 'name','verified', 'created_at', 'retweet_count', 'favorite_count', 'followers_count','hashtags', 'text']]
text_data = text_data.rename(columns = {'Unnamed: 0': 'tweet_id'})
text_data.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of..."
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...
4,4,Ian Heinisch,True,2022-03-05 14:45:25+00:00,45,149,7822,[],Make sure you check in on your bros in the str...


In [15]:
punctuation = string.punctuation
punctuation = punctuation.replace("#", "")
punctuation = punctuation.replace("@", "")
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in punctuation])
    return punctuationfree

In [16]:
def tokenization(text):
    tokens = re.split('\W+',text)
    return tokens

In [17]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leigh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\leigh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def clean_text(text):
    text['clean_text']= text['text'].apply(lambda x:remove_punctuation(x))
    text['clean_text']= text['clean_text'].apply(lambda x: x.lower())
    text['clean_text']= text['clean_text'].apply(lambda x: tokenization(x))
    text['clean_text']= text['clean_text'].apply(lambda x:remove_stopwords(x))
    text['clean_text']= text['clean_text'].apply(lambda x:lemmatizer(x))
    clean = text
    return clean

In [20]:
text_data = clean_text(text_data)
text_data.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of...","[open, ledger, trying, sneak, lot, money, cryp..."
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...,"[, crypto, market, today, market, capitalizati..."
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...,"[g7, country, eu, looking, way, stop, russia, ..."
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...,"[singapore, introduced, sanction, russian, cen..."
4,4,Ian Heinisch,True,2022-03-05 14:45:25+00:00,45,149,7822,[],Make sure you check in on your bros in the str...,"[make, sure, check, bros, struggle, never, kno..."


In [21]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12717 entries, 0 to 12716
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         12717 non-null  int64 
 1   name             12717 non-null  object
 2   verified         12717 non-null  bool  
 3   created_at       12717 non-null  object
 4   retweet_count    12717 non-null  int64 
 5   favorite_count   12717 non-null  int64 
 6   followers_count  12717 non-null  int64 
 7   hashtags         12717 non-null  object
 8   text             12717 non-null  object
 9   clean_text       12717 non-null  object
dtypes: bool(1), int64(4), object(5)
memory usage: 906.7+ KB


## Sentiment Analysis

In [22]:
def textblob_polarity(text):
    return TextBlob(text).sentiment.polarity

In [23]:
text_data['textblob_sentiment'] = text_data['clean_text'].astype('str').apply(textblob_polarity)

In [24]:
def textblob_sentiment_summary(text):
    sentiment = "Neutral"
    if text > 0 :
        sentiment = "Positive"
    elif text < 0 :
        sentiment = "Negative"
    return sentiment

In [25]:
text_data['textblob_summary'] = text_data['textblob_sentiment'].apply(lambda x: textblob_sentiment_summary(x))

In [26]:
text_data.head()

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of...","[open, ledger, trying, sneak, lot, money, cryp...",0.0,Neutral
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...,"[, crypto, market, today, market, capitalizati...",-0.2,Negative
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...,"[g7, country, eu, looking, way, stop, russia, ...",0.0,Neutral
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...,"[singapore, introduced, sanction, russian, cen...",0.0,Neutral
4,4,Ian Heinisch,True,2022-03-05 14:45:25+00:00,45,149,7822,[],Make sure you check in on your bros in the str...,"[make, sure, check, bros, struggle, never, kno...",0.5,Positive


In [27]:
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\leigh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
text_data['vader_sentiment_pos'] = (text_data['clean_text'].astype('str')).apply(lambda x: sid.polarity_scores(x)['pos'])
text_data['vader_sentiment_neg'] = (text_data['clean_text'].astype('str')).apply(lambda x: sid.polarity_scores(x)['neg'])
text_data['vader_sentiment_neu'] = (text_data['clean_text'].astype('str')).apply(lambda x: sid.polarity_scores(x)['neu'])
text_data['vader_sentiment_compound'] = (text_data['clean_text'].astype('str')).apply(lambda x: sid.polarity_scores(x)['compound'])

In [29]:
def vader_sentiment_summary(text):
    sentiment = "Neutral"
    if text >= 0.05 :
        sentiment = "Positive"
    elif text <= - 0.05 :
        sentiment = "Negative"
    return sentiment

In [30]:
text_data['vader_summary'] = text_data['vader_sentiment_compound'].apply(lambda x: vader_sentiment_summary(x))

In [31]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12717 entries, 0 to 12716
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweet_id                  12717 non-null  int64  
 1   name                      12717 non-null  object 
 2   verified                  12717 non-null  bool   
 3   created_at                12717 non-null  object 
 4   retweet_count             12717 non-null  int64  
 5   favorite_count            12717 non-null  int64  
 6   followers_count           12717 non-null  int64  
 7   hashtags                  12717 non-null  object 
 8   text                      12717 non-null  object 
 9   clean_text                12717 non-null  object 
 10  textblob_sentiment        12717 non-null  float64
 11  textblob_summary          12717 non-null  object 
 12  vader_sentiment_pos       12717 non-null  float64
 13  vader_sentiment_neg       12717 non-null  float64
 14  vader_

## Saving Analysis per Coin

In [32]:
bitcoin_sentiment = text_data[text_data['text'].str.contains('bitcoin|BTC')]
ethereum_sentiment = text_data[text_data['text'].str.contains('ethereum|ETH')]
doge_sentiment = text_data[text_data['text'].str.contains('dogecoin|DOGE')]
avalanche_sentiment = text_data[text_data['text'].str.contains('avalanche|AVAX')]
solana_sentiment = text_data[text_data['text'].str.contains('solana|SOL')]
crypto_sentiment = text_data[text_data['text'].str.contains('crypto|cryptocurrency|coin')]

In [33]:
bitcoin_sentiment.to_csv("bitcoin_sentiment.csv")
bitcoin_sentiment

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
15,15,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
23,23,Bitcoin News,True,2022-03-05 16:34:04+00:00,90,267,2437101,"[{'text': 'bitcoin', 'indices': [81, 89]}, {'t...",Shake Shack says participants will get 15% of ...,"[shake, shack, say, participant, get, 15, purc...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
34,34,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
42,42,Bitcoin News,True,2022-03-05 16:34:04+00:00,90,267,2437101,"[{'text': 'bitcoin', 'indices': [81, 89]}, {'t...",Shake Shack says participants will get 15% of ...,"[shake, shack, say, participant, get, 15, purc...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
57,57,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12701,755,Crypto Jack,True,Mon Apr 04 12:00:20 +0000 2022,76,396,31910,"[{'text': 'BTC', 'indices': [48, 52]}, {'text'...",Which Crypto project you believe strongly ?🚀\...,"[crypto, project, believe, strongly, btc, bnb,...",0.433333,Positive,0.0,0.0,1.0,0.0,Neutral
12702,756,Wirex,True,Mon Apr 04 07:00:29 +0000 2022,24,69,41535,"[{'text': 'BTC', 'indices': [108, 112]}]",Get industry best interest % just for HODLing ...,"[get, industry, best, interest, hodling, bag, ...",1.000000,Positive,0.0,0.0,1.0,0.0,Neutral
12705,759,May,True,Mon Apr 04 08:15:18 +0000 2022,1518,1605,158839,"[{'text': 'Upril', 'indices': [18, 24]}, {'tex...","Once again, Happy #Upril 😍\nShill me your proj...","[happy, upril, shill, project, month, btc, cry...",0.800000,Positive,0.0,0.0,1.0,0.0,Neutral
12706,760,LunarCrush,True,Mon Apr 04 02:08:30 +0000 2022,147,1102,259474,"[{'text': 'bitcoin', 'indices': [100, 108]}]",🤔Which coins had the highest number of individ...,"[, coin, highest, number, individual, social, ...",0.016667,Positive,0.0,0.0,1.0,0.0,Neutral


In [34]:
ethereum_sentiment.to_csv("ethereum_sentiment.csv")
ethereum_sentiment

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
15,15,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
34,34,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
57,57,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
72,72,OKX_Brasil (antiga OKEx),True,2022-03-05 00:00:58+00:00,17,54,4541,"[{'text': 'BTC', 'indices': [52, 56]}, {'text'...",Top 10 moedas mencionadas no Twitter esta sema...,"[top, 10, moedas, mencionadas, twitter, esta, ...",0.500000,Positive,0.0,0.0,1.0,0.0,Neutral
82,82,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12692,746,Crypto Jack,True,Mon Apr 04 12:00:20 +0000 2022,76,396,31910,"[{'text': 'BTC', 'indices': [48, 52]}, {'text'...",Which Crypto project you believe strongly ?🚀\...,"[crypto, project, believe, strongly, btc, bnb,...",0.433333,Positive,0.0,0.0,1.0,0.0,Neutral
12693,747,Crypto.com,True,Mon Apr 04 03:57:52 +0000 2022,45,239,1904211,"[{'text': 'ATOM', 'indices': [36, 41]}, {'text...","Easily manage 500+ coins, including #ATOM, #AA...","[easily, manage, 500, coin, including, atom, a...",0.433333,Positive,0.0,0.0,1.0,0.0,Neutral
12694,748,1inch Network,True,Mon Apr 04 13:08:18 +0000 2022,22,76,1016147,"[{'text': 'ETH', 'indices': [2, 6]}, {'text': ...",📣 #ETH burned weekly update 📣\n\n🦄🔥 #1inch sti...,"[, eth, burned, weekly, update, 1inch, still, ...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
12695,749,Crypto.com NFT,True,Mon Apr 04 14:00:02 +0000 2022,32,89,250075,"[{'text': 'ETH', 'indices': [49, 53]}]",We’ve just added the Brian NFT Collection to o...,"[added, brian, nft, collection, eth, whitelist...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral


In [35]:
doge_sentiment.to_csv("doge_sentiment.csv")
doge_sentiment

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
15,15,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
20,20,David Gokhshtein,True,2022-03-05 13:43:06+00:00,148,1536,610961,"[{'text': 'crypto', 'indices': [26, 33]}]",How many people came into #crypto because of $...,"[many, people, came, crypto, doge]",0.500000,Positive,0.0,0.0,1.0,0.0,Neutral
34,34,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
39,39,David Gokhshtein,True,2022-03-05 13:43:06+00:00,148,1536,610961,"[{'text': 'crypto', 'indices': [26, 33]}]",How many people came into #crypto because of $...,"[many, people, came, crypto, doge]",0.500000,Positive,0.0,0.0,1.0,0.0,Neutral
57,57,The Moon,True,2022-03-05 16:55:01+00:00,266,1776,1013020,"[{'text': 'crypto', 'indices': [12, 19]}]",Choose your #crypto team:\r\n\r\nTeam 🅰️ ...,"[choose, crypto, team, team, team, btc, eth, m...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12644,698,The Moon,True,Mon Apr 04 15:24:53 +0000 2022,72,704,1036863,"[{'text': 'DOGE', 'indices': [13, 18]}]",Should I buy #DOGE now?,"[buy, doge]",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
12645,699,SlumDOGE Millionaire,True,Mon Apr 04 15:17:42 +0000 2022,26,222,217565,"[{'text': 'dogecoin', 'indices': [40, 49]}]",I honestly feel like if we all take our #dogec...,"[honestly, feel, like, take, dogecoin, exchang...",0.600000,Positive,0.0,0.0,1.0,0.0,Neutral
12666,720,Crypto Jack,True,Mon Apr 04 12:00:20 +0000 2022,76,396,31910,"[{'text': 'BTC', 'indices': [48, 52]}, {'text'...",Which Crypto project you believe strongly ?🚀\...,"[crypto, project, believe, strongly, btc, bnb,...",0.433333,Positive,0.0,0.0,1.0,0.0,Neutral
12692,746,Crypto Jack,True,Mon Apr 04 12:00:20 +0000 2022,76,396,31910,"[{'text': 'BTC', 'indices': [48, 52]}, {'text'...",Which Crypto project you believe strongly ?🚀\...,"[crypto, project, believe, strongly, btc, bnb,...",0.433333,Positive,0.0,0.0,1.0,0.0,Neutral


In [36]:
solana_sentiment.to_csv("solana_sentiment.csv")
solana_sentiment

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
499,16,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\r\n\r\nTeam 🅰️ ...,"[pick, crypto, team, team, team, ax, mana, sol...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
514,31,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\r\n\r\nTeam 🅰️ ...,"[pick, crypto, team, team, team, ax, mana, sol...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
533,50,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\r\n\r\nTeam 🅰️ ...,"[pick, crypto, team, team, team, ax, mana, sol...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
553,70,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\r\n\r\nTeam 🅰️ ...,"[pick, crypto, team, team, team, ax, mana, sol...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
587,104,The Moon,True,2022-03-06 19:09:34+00:00,127,897,1013020,"[{'text': 'crypto', 'indices': [7, 14]}]",Pick a #crypto team:\r\n\r\nTeam 🅰️ ...,"[pick, crypto, team, team, team, ax, mana, sol...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12424,478,Crypto King,True,Fri Apr 01 04:50:26 +0000 2022,25,250,81276,"[{'text': 'Crypto', 'indices': [67, 74]}, {'te...",Yellow Gold or Rose Gold?\n\nAt 111 likes goin...,"[yellow, gold, rose, gold, 111, like, going, h...",0.300000,Positive,0.0,0.0,1.0,0.0,Neutral
12513,567,Raoul Pal,True,Sat Apr 02 00:27:48 +0000 2022,256,2116,919170,[],Crypto: \n\nThe SOL/ETH chart is super interes...,"[crypto, soleth, chart, super, interesting, so...",0.208333,Positive,0.0,0.0,1.0,0.0,Neutral
12708,762,WendyO.eth ✨Attending Bitcoin Miami 2022,True,Mon Apr 04 04:46:00 +0000 2022,30,235,268961,"[{'text': 'Solana', 'indices': [37, 44]}, {'te...",TY to @worldofsolana for my honorary #Solana #...,"[ty, worldofsolana, honorary, solana, nft, bel...",0.136364,Positive,0.0,0.0,1.0,0.0,Neutral
12711,765,fxnction aka igotabigdick.eth,True,Mon Apr 04 14:01:55 +0000 2022,6,239,96962,"[{'text': 'solana', 'indices': [18, 25]}]",Love finding sick #solana mints under the radar 👀,"[love, finding, sick, solana, mint, radar, ]",-0.107143,Negative,0.0,0.0,1.0,0.0,Neutral


In [37]:
avalanche_sentiment.to_csv("avalanche_sentiment.csv")
avalanche_sentiment

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
146,146,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,"[looking, altcoins, skyrocket, weekend, bsc, e...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
187,187,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,"[looking, altcoins, skyrocket, weekend, bsc, e...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
236,236,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,"[looking, altcoins, skyrocket, weekend, bsc, e...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
250,250,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,"[looking, altcoins, skyrocket, weekend, bsc, e...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
291,291,Túlio,True,2022-03-05 13:35:23+00:00,96,134,840626,"[{'text': 'altcoins', 'indices': [17, 26]}, {'...",Looking for some #altcoins that will skyrocket...,"[looking, altcoins, skyrocket, weekend, bsc, e...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10682,261,Bitcoin News,True,Wed Mar 30 13:02:55 +0000 2022,37,85,2481843,"[{'text': 'DeFiKingdoms', 'indices': [73, 86]}...",DeFi Kingdoms Integrates Ramp to Take in-Game ...,"[defi, kingdom, integrates, ramp, take, ingame...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
11258,78,WeatherNation,True,Mon Mar 28 14:00:00 +0000 2022,35,92,208922,[],Whoa! 🌨️⛰️ This is the result of an avalanche ...,"[whoa, result, avalanche, eagle, river, ak, ma...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
11441,261,Bitcoin News,True,Wed Mar 30 13:02:55 +0000 2022,37,85,2481843,"[{'text': 'DeFiKingdoms', 'indices': [73, 86]}...",DeFi Kingdoms Integrates Ramp to Take in-Game ...,"[defi, kingdom, integrates, ramp, take, ingame...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral
12024,78,WeatherNation,True,Mon Mar 28 14:00:00 +0000 2022,35,92,208922,[],Whoa! 🌨️⛰️ This is the result of an avalanche ...,"[whoa, result, avalanche, eagle, river, ak, ma...",0.0,Neutral,0.0,0.0,1.0,0.0,Neutral


In [38]:
crypto_sentiment.to_csv("crypto_sentiment.csv")
crypto_sentiment

,tweet_id,name,verified,created_at,retweet_count,favorite_count,followers_count,hashtags,text,clean_text,textblob_sentiment,textblob_summary,vader_sentiment_pos,vader_sentiment_neg,vader_sentiment_neu,vader_sentiment_compound,vader_summary
0,0,Bitcoin News,True,2022-03-05 09:33:07+00:00,151,520,2437101,"[{'text': 'crypto', 'indices': [61, 68]}]","""It is an open ledger, trying to sneak lots of...","[open, ledger, trying, sneak, lot, money, cryp...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
1,1,Bitcoin News,True,2022-03-05 12:03:14+00:00,87,245,2437101,"[{'text': 'crypto', 'indices': [5, 12]}]",“The #crypto market today has a market capital...,"[, crypto, market, today, market, capitalizati...",-0.200000,Negative,0.0,0.0,1.0,0.0,Neutral
2,2,Bitcoin News,True,2022-03-05 13:33:29+00:00,95,245,2437101,"[{'text': 'cryptocurrency', 'indices': [88, 10...",G7 countries and the EU are looking at ways to...,"[g7, country, eu, looking, way, stop, russia, ...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
3,3,Watcher.Guru,True,2022-03-05 11:01:13+00:00,149,1043,708057,"[{'text': 'cryptocurrency', 'indices': [77, 92]}]",JUST IN: 🇸🇬 Singapore has introduced sanctions...,"[singapore, introduced, sanction, russian, cen...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
5,5,Ian Heinisch,True,2022-03-05 21:34:37+00:00,16,27,7822,"[{'text': 'cryptocurrency', 'indices': [54, 69]}]",I can not wait for the move to earn app. @Yeti...,"[wait, move, earn, app, yeticoineth, cryptocur...",0.000000,Neutral,0.0,0.0,1.0,0.0,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12702,756,Wirex,True,Mon Apr 04 07:00:29 +0000 2022,24,69,41535,"[{'text': 'BTC', 'indices': [108, 112]}]",Get industry best interest % just for HODLing ...,"[get, industry, best, interest, hodling, bag, ...",1.000000,Positive,0.0,0.0,1.0,0.0,Neutral
12705,759,May,True,Mon Apr 04 08:15:18 +0000 2022,1518,1605,158839,"[{'text': 'Upril', 'indices': [18, 24]}, {'tex...","Once again, Happy #Upril 😍\nShill me your proj...","[happy, upril, shill, project, month, btc, cry...",0.800000,Positive,0.0,0.0,1.0,0.0,Neutral
12706,760,LunarCrush,True,Mon Apr 04 02:08:30 +0000 2022,147,1102,259474,"[{'text': 'bitcoin', 'indices': [100, 108]}]",🤔Which coins had the highest number of individ...,"[, coin, highest, number, individual, social, ...",0.016667,Positive,0.0,0.0,1.0,0.0,Neutral
12714,768,Wirex,True,Mon Apr 04 07:00:29 +0000 2022,24,69,41535,"[{'text': 'BTC', 'indices': [108, 112]}]",Get industry best interest % just for HODLing ...,"[get, industry, best, interest, hodling, bag, ...",1.000000,Positive,0.0,0.0,1.0,0.0,Neutral


In [39]:
len_doge = len(doge_sentiment)
len_btc = len(bitcoin_sentiment)
len_avax = len(avalanche_sentiment)
len_eth = len(ethereum_sentiment)
len_sol = len(solana_sentiment)

print(len(text_data)-len_doge-len_sol-len_eth-len_btc-len_avax)
print(len(text_data)-len_doge-len_sol-len_eth-len_btc-len_avax-len(crypto_sentiment))

7677
1411


Not all the tweets pertain to the coins - there is clearly some error within the API as there are some tweets with no hashtags or mention of the specified criteria. Perhaps it pulled in some viral tweets that were important across all spaces despite not explicitly mentioning the keywords.